In [ ]:
%pylab notebook

In [ ]:
# To get animation working
import matplotlib.animation
from IPython.display import HTML

In [ ]:
# Define colorize(z) to do Complex 2D Magnitude/Phase plots
# Copy and Paste this to plot complex 2D arrays
# See https://en.wikipedia.org/wiki/Domain_coloring
# from https://stackoverflow.com/questions/17044052/mathplotlib-imshow-complex-2d-array
from colorsys import hls_to_rgb

def colorize(z, log=False):
    '''
    Turn the array z of complex numbers into an array of rgb values where
    the hue is determined by the phase and the lightness is determined by
    the magnitude or the log of the magnitude
    '''
    n,m = z.shape
    c = np.zeros((n,m,3))
    # Infinity and non-a-number entries will be turned into something that won't throw errors
    c[np.isinf(z)] = (1.0, 1.0, 1.0)
    c[np.isnan(z)] = (0.5, 0.5, 0.5)
    
    idx = ~(np.isinf(z) + np.isnan(z))  # indicies that contain finite numbers
    # First get the hue based on the phase (angle) of the complex number
    A = (np.angle(z[idx]) + np.pi) / (2*np.pi)
    A = (A + 0.5) % 1.0
    # Now get the lightness based on the magnitude or log(magnitude)
    if log:
        #v = np.arctan(np.log(np.abs(z[idx])))/np.pi+0.5  # goes from 0 to 1 when log(abs(z)) goes -inf to +inf
        v = np.log(np.abs(z[idx]))
        # Rescale to the middle 99.9 percentile and clip off the outliers
        low  = np.percentile(v, 1)
        high = np.percentile(v,99.99)
        B = (v-low)/(high-low)  # rescale
        B[B<0] = 0.0
        B[B>1] = 1.0
        print(np.min(B), np.max(B))
    else:
        #B = 1.0 - 1.0/(1.0+abs(z[idx])**0.3)
        B = 1.0 - 1.0/(1.0+abs(z[idx])**0.6)  # the exponent must be less than 0
        #B = 2/np.pi*np.arctan(abs(z[idx]))
    c[idx] = [hls_to_rgb(a, b, 0.8) for a,b in zip(A,B)]
    return c

In [ ]:
# Gaussian Beam
lam = 1.0
w0 = 0.7
#w0 = sqrt(lam*z0/pi) # Rayleigh length

def w(z,lam,w0):
    'beam waist as a function of z'
    z0 = pi*w0**2/lam
    return w0*sqrt(1+(z/z0)**2)
def R(z,w0):
    'radius of curvature of the wave fronts'
    z0 = pi*w0**2/lam
    return z*(1+(z0/z)**2)
def envelope(r,z,lam,w0):
    k = 2*pi/lam
    z0 = pi*w0**2/lam
    amplitude = w0/w(z,lam,z0) * exp(-r**2/w(z,lam,z0)**2)
    longitudinal_phase = -arctan(z/z0)
    radial_phase = k*r**2/(2*R(z,z0))
    return amplitude*exp(1j*longitudinal_phase + 1j*radial_phase)
def Eplus(r,z,lam,w0):
    k = 2*pi/lam
    return envelope(r,z,lam,w0)*exp(1j*k*z)

axis_x = np.linspace(-18,18,180*4)
axis_y = np.linspace(-10,10,100*4)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
A = Eplus(Y,X,lam,w0)

# Plot the array "A" using colorize
figure(figsize=(9,5))
extents = (min(axis_x),max(axis_x),min(axis_y),max(axis_y))
imshow(colorize(A), interpolation='none',extent=extents,origin='lower')
title('Gaussian Beam with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

In [ ]:
A = envelope(Y,X,lam,w0)

# Plot the array "A" using colorize
figure(figsize=(9,5))
extents = (min(axis_x),max(axis_x),min(axis_y),max(axis_y))
imshow(colorize(A), interpolation='none',extent=extents,origin='lower')
title('Just $\psi$ for Gaussian Beam Envelope with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

In [ ]:
# Intensity
figure(figsize=(9,5))
extents = (min(axis_x),max(axis_x),min(axis_y),max(axis_y))
imshow(abs(A)**2, interpolation='none',extent=extents,origin='lower',cmap='gray')
#colorbar()
title('Gaussian Beam Intensity with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

In [ ]:
# Animate Gaussian beams of different size
t = np.linspace(0.3,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
A = Eplus(Y,X,lam,w0)
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
title('Gaussian Beams with increasing beam waists $w_0$')
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    w0 = t[i]
    A = Eplus(Y,X,lam,w0)
    im.set_data(colorize(A))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
# Animate just Psi for different beam sizes
t = np.linspace(0.3,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
A = 0*Eplus(Y,X,lam,w0)
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
title('Just $\psi$ for Gaussian Envelope with increasing beam waists $w_0$')
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    w0 = t[i]
    A = envelope(Y,X,lam,w0)
    im.set_data(colorize(A))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
# Animate intensity for different beam sizes
t = np.linspace(0.3,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
A = 0*Eplus(Y,X,lam,w0)
im = ax.imshow(abs(A)**2,extent=extents,origin='lower',cmap='gray',vmin=0,vmax=0.3)  # Need to spcify vmin,vmax
title('Gaussian Intensity with increasing beam waists $w_0$')
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    w0 = t[i]
    A = Eplus(Y,X,lam,w0)
    im.set_data(abs(A)**2)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
# Animate Right-Going Traveling Complex Gaussian Beam
t = np.linspace(0,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
w0 = 0.7
A = 0*X
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
title('Right-Going Traveling Complex Gaussian Beam with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    A = Eplus(Y,X,lam,w0)*exp(-2j*pi*t[i])
    im.set_data(colorize(A))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
# Animate Real Right-Going Traveling Gaussian
t = np.linspace(0,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
w0 = 0.7
A = 0*X
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
title('Real Right-Going Traveling Gaussian Beam with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    A = Eplus(Y,X,lam,w0)*exp(-2j*pi*t[i])
    R = A + conjugate(A)
    im.set_data(colorize(R))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
# Animate Real Left-Going Traveling Gaussian Beam
t = np.linspace(0,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
w0 = 0.7
A = 0*X
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
title('Real Left-Going Traveling Gaussian Beam with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    A = conjugate(Eplus(Y,X,lam,w0))*exp(-2j*pi*t[i])
    L = A + conjugate(A)
    im.set_data(colorize(L))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
# Animate Real Standing Gaussian Beam
t = np.linspace(0,1,num=10)  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
w0 = 0.7
A = 0*X
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
title('Real Standing Gaussian Beam with $w_0=%0.1f\lambda$'%w0);
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    R = Eplus(Y,X,lam,w0)*exp(-2j*pi*t[i])
    R = R + conjugate(R)
    A = conjugate(Eplus(Y,X,lam,w0))*exp(-2j*pi*t[i])
    L = A + conjugate(A)
    im.set_data(colorize(L+R))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
# Animate Mirrors with Real Standing Gaussian Beam
t = np.arange(16)/16.0  # time steps
fig, ax = plt.subplots(figsize=(9,5))
axis_x = np.linspace(-18,18,180*2)
axis_y = np.linspace(-10,10,100*2)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
w0 = 0.7
A = 0*X
im = ax.imshow(colorize(A),extent=extents,origin='lower')  # Need to spcify vmin,vmax
mirror = 17
title('Mirrors at $\pm%0.1f\lambda$ for Real Standing Gaussian Beam with $w_0=%0.1f\lambda$'%(mirror,w0));
xlabel('z (units of $\lambda$)')
ylabel('x or y (units of $\lambda$)');

def animate(i):
    A = Eplus(Y,X,lam,w0)*exp(-2j*pi*t[i])
    R = A + conjugate(A)
    A = conjugate(Eplus(Y,X,lam,w0))*exp(-2j*pi*t[i])
    L = A + conjugate(A)
    mask = (X**2 + Y**2) < (mirror)**2
    im.set_data(colorize((L+R)*mask))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

In [ ]:
HTML(ani.to_jshtml())

erf?

In [ ]:
import scipy.special

In [ ]:
scipy.special.erf(1)

In [ ]:
scipy.special.erfinv(0.843)

In [ ]:
axis_x = np.linspace(-3,3,500)
axis_y = np.linspace(-3,3,500)
X,Y = np.meshgrid(axis_x,axis_y)  # Two 2D arrays, which change either in x or y directions
A = exp(-X**2-Y**2) * (X>-0.5)

# Plot the array "A" using colorize
figure()
extents = (min(axis_x),max(axis_x),min(axis_y),max(axis_y))
imshow(A,cmap='gray',interpolation='none',extent=extents,origin='lower')
title('Gaussian profile with knife edge at x=-0.5');
xlabel('x [arbitrary units]')
ylabel('y [arbitrary units]');
tight_layout();
savefig('knife_edge_gaussian.pdf');